Web scrapping allrecipes.com

Nombres: Wilson García y Doménica Plaza

In [2]:
from bs4 import BeautifulSoup


In [29]:
file = 'data/receta.html'

with open(file, 'r', encoding='utf-8') as f:
    html_content = f.read()
    
soup = BeautifulSoup(html_content, 'html.parser')

Descargar otras paginas

In [32]:
recipes_links = soup.find_all("a")
recipe_urls = []
for link in recipes_links:
    href = link["href"]
    if "recipe" in href:
        recipe_urls.append(href)
        
print("Linked recipes:")
for url in recipe_urls:
    print(url)
print(len(recipe_urls))


Linked recipes:
https://www.allrecipes.com/authentication/login?regSource=3675&relativeRedirectUrl=%2Frecipe%2F18074%2Fmarinated-flank-steak%2F
/account/add-recipe
https://www.myrecipes.com/favorites
https://www.allrecipes.com/authentication/logout?relativeRedirectUrl=%2Frecipe%2F18074%2Fmarinated-flank-steak%2F
https://www.magazines.com/allrecipes-magazine.html?utm_source=allrecipes.com&utm_medium=owned&utm_campaign=i111arr1w2661
https://www.magazines.com/allrecipes-magazine.html
https://www.allrecipes.com/recipes/17562/dinner/
https://www.allrecipes.com/recipes/17057/everyday-cooking/more-meal-ideas/5-ingredients/main-dishes/
https://www.allrecipes.com/recipes/15436/everyday-cooking/one-pot-meals/
https://www.allrecipes.com/recipes/1947/everyday-cooking/quick-and-easy/
https://www.allrecipes.com/recipes/455/everyday-cooking/more-meal-ideas/30-minute-meals/
https://www.allrecipes.com/recipes/17889/everyday-cooking/family-friendly/family-dinners/
https://www.allrecipes.com/recipes/94/s

Bola de Nieve (Scraping)

In [5]:
title = soup.find("meta", {"property": "og:title"})["content"]
print(title)

Marinated Flank Steak


In [6]:
ingredients_section = soup.find_all("li", class_='mm-recipes-structured-ingredients__list-item')
for ingredient in ingredients_section:
    print(ingredient.text.strip())

½ cup vegetable oil
⅓ cup low-sodium soy sauce
¼ cup red wine vinegar
2 tablespoons fresh lemon juice
1 ½ tablespoons Worcestershire sauce
1 tablespoon Dijon mustard
2 cloves garlic, minced
½ teaspoon ground black pepper
1 (1 1/2-pound) flank steak


In [11]:
rating = soup.find("div", class_="comp mm-recipes-review-bar__rating mntl-text-block text-label-300").text.strip()
print(rating)

4.7


In [9]:
# Busca todos los bloques de tiempo
items = soup.find_all("div", class_="mm-recipes-details__item")

# Recorre cada uno buscando el que tenga la etiqueta "Total Time:"
for item in items:
    label = item.find("div", class_="mm-recipes-details__label")
    value = item.find("div", class_="mm-recipes-details__value")   
    if label and "Total Time:" in label.get_text(strip=True):
        total_time = value.get_text(strip=True)
        print("Total Time:", total_time)
    if label and "Servings" in label.get_text(strip=True):
        servings = value.get_text(strip=True)
        print("Servings:", servings)
        break
        


Total Time: 2 hrs 25 mins
Servings: 6


In [ ]:
#summary
# Buscar todos los bloques de texto
paragraphs = soup.find_all("p")

for p in paragraphs:
    text = p.get_text(strip=True)
    if len(text) > 100:  
        summary = text
        break

print(summary)


A great flank steak marinade like this one is important if you want a tender, juicy, flavorful steak. Make sure you marinate your flank steak for at least 2 hours for best results or longer if you have time. This wonderful quick and easy recipe also works great when the steak is sliced and used for fajitas.


In [13]:
# Extraer pasos de preparación sin créditos
preparation_steps = []

step_items = soup.select("li.mntl-sc-block-group--LI")

for i, item in enumerate(step_items, 1):
    text = item.get_text(" ", strip=True)
    if text and len(text) > 20:
        # Eliminar texto no deseado si está al final
        if text.endswith("Dotdash Meredith Food Studios"):
            text = text.replace("Dotdash Meredith Food Studios", "").strip()
        preparation_steps.append(f"step {i}: {text}")

# Imprimir los pasos limpios
for step in preparation_steps:
    print(step)

step 1: Gather all ingredients.
step 2: Whisk together oil, soy sauce, vinegar, lemon juice, Worcestershire sauce, Dijon mustard, garlic, and pepper for marinade until thoroughly combined. Place steak in a 9x13-inch glass baking dish.
step 3: Pour marinade over flank steak in the baking dish; turn several times to coat thoroughly with marinade. Cover, and refrigerate for 2 to 6 hours, or up to 12 hours if you have time.
step 4: When ready to cook, preheat an outdoor grill for medium-high heat and lightly oil the grate.
step 5: Remove steak from the marinade and shake off excess. Discard the remaining marinade.
step 6: Cook steak on the preheated grill for about 5 minutes per side, or to desired doneness.
step 7: Remove from the grill and let rest for 5 minutes before slicing and serving.
step 8: Serve hot and enjoy! DOTDASH MEREDITH FOOD STUDIOS


In [14]:
# Buscar las filas de la tabla de nutrición
rows = soup.select("tr.mm-recipes-nutrition-facts-summary__table-row")

nutrition_facts = {}
for row in rows:
    columns = row.find_all("td")
    if len(columns) >= 2:
        label = columns[1].get_text(strip=True)
        value = columns[0].get_text(strip=True)
        nutrition_facts[label] = value

# Imprimir resultados
for k, v in nutrition_facts.items():
    print(f"{k}: {v}")

Calories: 363
Fat: 28g
Carbs: 4g
Protein: 25g


In [20]:
import pandas as pd
import ace_tools_open as tools
# Create DataFrame
df = pd.DataFrame([{
    "title": title,
    "summary": summary,
    "ingredients": ingredients_section,
    "steps": preparation_steps,
    "total_time": total_time,
    "servings": servings,
    "rating": rating,
    "nutrition": nutrition_facts
}])


tools.display_dataframe_to_user(name="Recetas", dataframe=df)

Recetas


Loading ITables v2.4.4 from the internet... (need help?)


Para 100 links

In [23]:
import requests
import time
from bs4 import BeautifulSoup

recipe_data = []

headers = {"User-Agent": "Mozilla/5.0"}

for url in recipe_urls:
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        title = soup.find("meta", {"property": "og:title"})
        title = title["content"] if title else "Título no encontrado"

        summary = next(
            (p.get_text(strip=True) for p in soup.find_all("p") if len(p.get_text(strip=True)) > 100),
            "Resumen no encontrado"
        )

        ingredients = " | ".join([
            ing.text.strip()
            for ing in soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
        ]) or "Ingredientes no encontrados"

        steps = " ".join([
            f"step {i+1}: {item.get_text(' ', strip=True)}"
            for i, item in enumerate(soup.select("li.mntl-sc-block-group--LI"))
        ]) or "Pasos no encontrados"

        total_time = next(
            (value.text.strip() for label, value in zip(
                soup.find_all("div", class_="mm-recipes-details__label"),
                soup.find_all("div", class_="mm-recipes-details__value"))
             if "Total Time" in label.text),
            "Tiempo no encontrado"
        )

        servings = next(
            (value.text.strip() for label, value in zip(
                soup.find_all("div", class_="mm-recipes-details__label"),
                soup.find_all("div", class_="mm-recipes-details__value"))
             if "Servings" in label.text),
            "Porciones no encontradas"
        )

        rating_tag = soup.find("div", class_="comp mm-recipes-review-bar__rating mntl-text-block text-label-300")
        rating = rating_tag.text.strip() if rating_tag else "Rating no encontrado"

        nutrition = " | ".join([
            f"{row.find_all('td')[1].get_text(strip=True)}: {row.find_all('td')[0].get_text(strip=True)}"
            for row in soup.select("tr.mm-recipes-nutrition-facts-summary__table-row")
            if len(row.find_all('td')) >= 2
        ]) or "Nutrición no encontrada"

        # Agregar a la lista
        recipe_data.append({
            "url": url,
            "title": title,
            "summary": summary,
            "ingredients": ingredients,
            "steps": steps,
            "total_time": total_time,
            "servings": servings,
            "rating": rating,
            "nutrition": nutrition
        })

        print(f"✅ Receta extraída: {title}")

    except Exception as e:
        print(f"❌ Error en {url}: {e}")

    time.sleep(1)  # evitar sobrecarga

# Mostrar un ejemplo
print("\nEjemplo de receta extraída:")
print(recipe_data[0])


✅ Receta extraída: Título no encontrado
❌ Error en /account/add-recipe: Invalid URL '/account/add-recipe': No scheme supplied. Perhaps you meant https:///account/add-recipe?
✅ Receta extraída: Título no encontrado
✅ Receta extraída: Marinated Flank Steak
✅ Receta extraída: Allrecipes Magazine Subscription | Subscribe to Allrecipes Magazine
✅ Receta extraída: Allrecipes Magazine Subscription | Subscribe to Allrecipes Magazine
✅ Receta extraída: Dinner Recipes
✅ Receta extraída: 5 Ingredient Main Dishes
✅ Receta extraída: One-Pot Meal Recipes
✅ Receta extraída: Quick and Easy Recipes
✅ Receta extraída: 30-Minute Meals
✅ Receta extraída: Family Dinner Recipes
✅ Receta extraída: Soups, Stews and Chili Recipes
✅ Receta extraída: Comfort Food
✅ Receta extraída: Main Dishes
✅ Receta extraída: Sheet Pan Dinner Recipes
✅ Receta extraída: Dinner Recipes
✅ Receta extraída: Recipes A-Z
✅ Receta extraída: Breakfast and Brunch Recipes
✅ Receta extraída: Lunch Recipes
✅ Receta extraída: Healthy Recip

In [24]:
df = pd.DataFrame(recipe_data)
tools.display_dataframe_to_user(name="Recetas", dataframe=df)

Recetas


Loading ITables v2.4.4 from the internet... (need help?)
